In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# pages of main url
url = 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, features="lxml")

max_page = soup.findAll('a', class_="v-A_-button v-A_-button--medium v-A_-button--mediumgrey")[-1].text
max_page = float(max_page.strip())
max_page

pages = []
pages.append(url)
for i in range(int(max_page)):
    url = 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=' + str(i+1)
    pages.append(url)
pages

['https://www.runnersworld.de/volkslaeufe-strassenlaeufe/',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=1',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=2',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=3',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=4',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=5',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=6',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=7',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=8',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=9',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=10',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=11',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=12',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=13',
 'https://www.runnersworld.de/volkslaeufe-strassenlaeufe/?p=14',
 'https://www.runnersworld.de/volkslaeu

## A single run

In [81]:
url = "https://www.runnersworld.de/laufkalender/johanniterlaufleuggern/"
url = "https://www.runnersworld.de/laufkalender/5-k-race-day-refrath/"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)

In [89]:
name = soup.find('span', class_='v-A_-headline v-A_-headline__article--alpha').text
name

'5k Race Day Refrath'

In [90]:
# date of run
date = soup.find('div', class_='v-A_-appointment').find('span').text
date

'26.05.2022'

In [91]:
# website
website = soup.find('div', class_='v-A_-information v-A_-connection').find('a').text
website

'www.5k-raceday.de'

In [92]:
location = soup.find('div', class_='v-A_-information v-A_-location').find('span').text
location = location.strip('\n')
location

'51427  Bergisch Gladbach, Willhelm-Klein-Str 18-20 (Refrath)'

In [105]:
def scrape_table(soup):
    table = soup.find('table')

    rows = [] #make list for rows
    for row in table.findAll('tr'):
        list_cells = [] # array for row
        for cell in row.findAll('td'):
            text = cell.text.replace('&nbsp;', '') #replace nonbreaking space
            text = text.strip()
            list_cells.append(text)
        if len(list_cells) > 0:
            rows.append(list_cells)
    return rows

tbl = scrape_table(soup=soup)
tbl

[['5 km Männer', '13.15 Uhr', '15,00 - 18,00 Euro'],
 ['5 km Frauen', '13.45 Uhr', '15,00 - 18,00 Euro']]

In [96]:
df = pd.DataFrame(tbl, columns=['distance', 'start_time', 'price'])
df.to_dict
df

,distance,start_time,price
0,5 km Männer,13.15 Uhr,"15,00 - 18,00 Euro"
1,5 km Frauen,13.45 Uhr,"15,00 - 18,00 Euro"


In [103]:
# df['distance'] = df['distance'].str.replace(',', '.')
# df['distance'] = df['distance'].str.replace(' km', '')
# # df['distance'] = df['distance'].astype(float)
df.to_dict()


{'distance': {0: '5 Männer', 1: '5 Frauen'},
 'start_time': {0: '13.15 Uhr', 1: '13.45 Uhr'},
 'price': {0: '15,00 - 18,00 Euro', 1: '15,00 - 18,00 Euro'}}

In [ ]:
currency = df.price[0].split(' ')[1]

df['price'] = df['price'].str.replace(' ' + currency, '')
df['price'] = df['price'].str.replace(',', '.')
df['currency'] = currency
df

,distance,start_time,price,currency
0,6.50,10.00 Uhr,25.00,CHF
1,1.06,15.00 Uhr,0.00,CHF
2,0.53,ab 13.40 Uhr,0.00,CHF
3,0.30,ab 13.00 Uhr,0.00,CHF


In [ ]:
min_distance, max_distance = df.distance.min(), df.distance.max()
min_distance, max_distance

(0.3, 6.5)

In [ ]:
min_price, max_price = df.price.min(), df.price.max()
min_price, max_price


('0.00', '25.00')

In [106]:
data_run = {
    'name' : name,
    'date' : date,
    'website' : website,
    'location' : location,
    'run_info' : tbl
    # 'min_price' : min_price,
    # 'max_price' : max_price,
    # 'currency' : currency,
    # 'min_distance' : min_distance,
    # 'max_distance' : max_distance
}

data_run

{'name': '5k Race Day Refrath',
 'date': '26.05.2022',
 'website': 'www.5k-raceday.de',
 'location': '51427  Bergisch Gladbach, Willhelm-Klein-Str 18-20 (Refrath)',
 'run_info': [['5 km Männer', '13.15 Uhr', '15,00 - 18,00 Euro'],
  ['5 km Frauen', '13.45 Uhr', '15,00 - 18,00 Euro']]}

## Multiple runs